## **Neel Patel-Data Preprocessing & Cleaning**


Tasks:
* Load the dataset (movies.csv).
* Remove duplicate entries.
* Handle missing values, convert necessary fields to numeric types, and re-check for NaNs.
* Standardize text fields, such as movie titles and genres, to ensure uniform formatting.
* Filter scores to ensure all ratings are within a valid range (0 to 10).
* Print out the number of missing values before and after cleaning.

Deliverables:
* Cleaned movies_df DataFrame, ready for Q-learning.
Output with missing values before and after cleaning.

In [1]:
import pandas as pd

# Load the dataset
movies_df = pd.read_csv('movies.csv')

# Step 1: Remove duplicates
movies_df.drop_duplicates(inplace=True)

# Step 2: Handle missing values
print("Missing values before cleaning:")
print(movies_df.isnull().sum())

# Drop rows with nulls in essential columns and convert 'score' to numeric
movies_df.dropna(subset=['name', 'genre', 'score'], inplace=True)
movies_df['score'] = pd.to_numeric(movies_df['score'], errors='coerce')
movies_df.dropna(inplace=True)

# Show the number of missing values after cleaning
print("\nMissing values after cleaning:")
print(movies_df.isnull().sum())

# Step 3: Standardize text fields and filter score range
movies_df['name'] = movies_df['name'].str.title()
movies_df['genre'] = movies_df['genre'].str.strip().str.capitalize()
movies_df = movies_df[(movies_df['score'] >= 0) & (movies_df['score'] <= 10)]

# Deliverable: Cleaned 'movies_df' DataFrame ready for Q-learning setup

Missing values before cleaning:
name           0
rating        77
genre          0
year           0
released       2
score          3
votes          3
director       0
writer         3
star           1
country        3
budget      2171
gross        189
company       17
runtime        4
dtype: int64

Missing values after cleaning:
name        0
rating      0
genre       0
year        0
released    0
score       0
votes       0
director    0
writer      0
star        0
country     0
budget      0
gross       0
company     0
runtime     0
dtype: int64


## **Nachiket Prajapati: Q-learning Setup & Training**

Tasks:
* Define states (genres) and actions (movie names) from the movies_df.
* Initialize the Q-table with zero values.
* Create a reward function based on movie ratings (e.g., positive reward for scores 7 and above, neutral for 4-6.9, negative for below 4).
* Implement the Q-learning training loop:
* Randomly select a genre (state) for each episode.
* Implement exploration-exploitation trade-off with epsilon-greedy strategy.
* Update the Q-table values using the Q-learning formula.
* Train for 500 episodes.

Deliverables:

* Trained Q-table with updated Q-values for state-action pairs.
* Code for Q-learning loop with detailed comments explaining each step.

In [2]:
import numpy as np
import pandas as pd
import random

# Load the cleaned 'movies_df' DataFrame from Neel's code output
# Define states (genres) and actions (movie names)
states = movies_df['genre'].unique().tolist()
actions = movies_df['name'].unique().tolist()

# Initialize Q-table with float type to avoid incompatible dtype warnings
q_table = pd.DataFrame(0.0, index=states, columns=actions)

# Define a mock reward function (simulating user feedback)
def get_reward(movie_name):
    movie_score = movies_df[movies_df['name'] == movie_name]['score'].mean()
    if movie_score >= 7.0:
        return 1  # Positive reward for scores 7 and above
    elif movie_score >= 4.0:
        return 0  # Neutral reward for scores between 4 and 6.9
    else:
        return -1  # Negative reward for scores below 4.0

# Q-learning parameters
alpha = 0.1  # Learning rate
gamma = 0.6  # Discount factor
epsilon = 0.1  # Exploration rate

# Q-learning algorithm
for episode in range(500):  # Number of training episodes
    state = random.choice(states)  # Choose a random genre as the user's preference

    done = False
    while not done:
        # Exploration-exploitation trade-off
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # Explore: recommend a random movie
        else:
            action = q_table.loc[state].idxmax()  # Exploit: choose best-known movie

        # Get reward based on the movie's score
        reward = get_reward(action)

        # Update Q-value
        old_value = q_table.loc[state, action]
        next_max = q_table.loc[state].max()
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table.loc[state, action] = new_value

        # End the loop if the recommendation is positive
        if reward > 0:
            done = True

# Deliverable: Trained 'q_table' with updated Q-values for state-action pairs

## **Priyankkumar Patel: User Interaction & Genre Selection**

Tasks:

* Display the available genres to the user with numbered options.
* Accept user input to choose a genre based on a number.
* Handle invalid inputs (e.g., numbers outside the genre list or non-numeric input).
* Pass the user-selected genre to the recommendation function.

Deliverables:

* Code for displaying genres, handling user input, and validating the selection.
* A variable user_preference containing the selected genre to be used by the recommendation function.

In [4]:
# Load the trained 'q_table' and 'states' list from Nachiket's output

# Display available genres with numbered options
print("Available genres:")
for idx, genre in enumerate(states, 1):
    print(f"{idx}. {genre}")

# Take user input as a number to select the genre
try:
    user_choice = int(input("\nEnter the number corresponding to your preferred genre: "))
    if 1 <= user_choice <= len(states):
        user_preference = states[user_choice - 1]
    else:
        print("Invalid selection. Please select a valid number.")
except ValueError:
    print("Please enter a valid number.")

# Deliverable: Selected 'user_preference' variable to be used for recommendations

Available genres:
1. Drama
2. Adventure
3. Action
4. Comedy
5. Horror
6. Biography
7. Crime
8. Fantasy
9. Animation
10. Family
11. Western
12. Sci-fi
13. Romance
14. Thriller
15. Mystery

Enter the number corresponding to your preferred genre: 12


## **Tirth Patel: Recommendation Logic & Output Formatting**

Tasks:

* Implement the recommend_top_movies function to:
* Filter movies within the selected genre with ratings between 6 and 10.
* Sort and select the top 5 unique movies based on Q-values.

Format and display recommendations:
* Ensure each recommendation includes the movie title, genre, and integer rating.
* Sort recommendations in descending order by rating.
* Add spacing and separators for readability.

In [6]:
# Load 'movies_df', 'q_table', and 'user_preference' from previous outputs

# Function to recommend top 5 unique movies with genre and integer ratings between 6 and 10
def recommend_top_movies(genre_preference, top_n=5):
    # Filter movies by the selected genre and ratings between 6 and 10
    genre_movies = movies_df[(movies_df['genre'] == genre_preference) & (movies_df['score'] >= 6) & (movies_df['score'] <= 10)]

    # Get the top N movies from the Q-table only for movies within the selected genre and rating range
    genre_movie_names = genre_movies['name'].tolist()
    filtered_q_values = q_table.loc[genre_preference, genre_movie_names]

    # Select the top N movies based on Q-values
    top_movies = filtered_q_values.nlargest(top_n * 2).index.tolist()  # Take extra to filter duplicates
    movie_recommendations = []

    # Retrieve genre and rating for each recommended movie
    for movie in top_movies:
        if movie not in [m[0] for m in movie_recommendations]:  # Check for uniqueness
            movie_data = genre_movies[genre_movies['name'] == movie].iloc[0]
            genre = movie_data['genre']
            rating = int(round(movie_data['score']))  # Convert rating to an integer
            movie_recommendations.append((movie, genre, rating))

        # Stop once we have the required number of unique recommendations
        if len(movie_recommendations) >= top_n:
            break

    # Sort recommendations by rating in descending order
    movie_recommendations = sorted(movie_recommendations, key=lambda x: x[2], reverse=True)
    return movie_recommendations

# Get top 5 unique recommendations for the selected genre with ratings between 6 and 10
recommended_movies = recommend_top_movies(user_preference, top_n=5)

# Display recommendations with genre and integer ratings
print(f"\nTop 5 recommended movies for '{user_preference}' lovers (Ratings 6-10):\n")
for idx, (movie, genre, rating) in enumerate(recommended_movies, 1):
    print(f"{idx}. {movie} | Genre: {genre} | Rating: {rating}")
    print("   " + "-"*50)

# Deliverable: Displayed list of top 5 recommendations


Top 5 recommended movies for 'Sci-fi' lovers (Ratings 6-10):

1. Limitless | Genre: Sci-fi | Rating: 7
   --------------------------------------------------
2. Jekyll And Hyde... Together Again | Genre: Sci-fi | Rating: 6
   --------------------------------------------------
3. Brainstorm | Genre: Sci-fi | Rating: 6
   --------------------------------------------------
4. The Manhattan Project | Genre: Sci-fi | Rating: 6
   --------------------------------------------------
5. Hardware | Genre: Sci-fi | Rating: 6
   --------------------------------------------------
